In [5]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json

In [6]:
#Reading Data, performing Sentiment on the data
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]
data['text'] = data['text'].apply(lambda x: x.lower())

#Obtaining the size of Positive and Negtive data
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
print(data[data['sentiment'] == 'Positive'].size)
print(data[data['sentiment'] == 'Negative'].size)

4472
16986


In [7]:
#Splitting the data and using Tokenizer
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)
#Printing the matrix
print(X)

[[   0    0    0 ...   51 1039    1]
 [   0    0    0 ... 1577 1356  847]
 [   0    0    0 ...   10  696  518]
 ...
 [   0    0    0 ...   68   62    3]
 [   0    0    0 ... 1112 1588   81]
 [   0    0    0 ...  196    3  880]]


In [8]:
embed_dim = 128
lstm_out = 196

#Creating the model
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [9]:
#Performing Transformation
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

#Obtaining XTrain, X Test, Y Train and Y Test
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [11]:
batch_size = 32
#Creating the model
model = createmodel()
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
291/291 - 45s - loss: 0.8242 - accuracy: 0.6442
Epoch 2/5
291/291 - 42s - loss: 0.6769 - accuracy: 0.7099
Epoch 3/5
291/291 - 42s - loss: 0.6127 - accuracy: 0.7450
Epoch 4/5
291/291 - 43s - loss: 0.5638 - accuracy: 0.7615
Epoch 5/5
291/291 - 43s - loss: 0.5236 - accuracy: 0.7781


In [12]:
#Obtaining the Accuracy and loss score
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

144/144 - 3s - loss: 0.8093 - accuracy: 0.6684
0.809275209903717
0.6684141755104065
['loss', 'accuracy']


In [13]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
#Extracting the model
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [15]:
#Predicting the Result
twt = ['A lot of good things are happening. We are respected again throughout the world, and thats a great thing.@realDonaldTrump' ]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)

[[  0   0   0   0   0   0   0   0   0   0   0   7 445   5 146 292  35  30
   35 371   2 349   8 262   7 153 265  23]]


In [16]:
#Predicting the sentiment
sentiment = loaded_model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)

1/1 - 0s
[0.3784042  0.04800899 0.57358676]


In [17]:
#Preforming the prediction
loaded_model.predict(twt,batch_size=1,verbose = 2)

1/1 - 0s


array([[0.3784042 , 0.04800899, 0.57358676]], dtype=float32)